In [1]:
import sys
sys.path.append("C:\Program Files\Tinysoft\Analyse.NET")
sys.path.append(r"C:\Users\xudong\Documents\github\coresearch\funcs")
sys.path.append(r"C:\Users\xudong\Documents\guanzhao\dataserver")
import pymongo
import numpy as np
import pickle
import time
import datetime
import copy
import pandas as pd
from operator import itemgetter
import TSLPy3 as tsl
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import talib
import importlib
import rschLib
from pylab import mpl
from library.tags import tags
np.set_printoptions(formatter={'float_kind': "{:.6f}".format})

db = rschLib.db_quanLiang()
dbt = rschLib.db_tinySoftData()
dtes, tkrs, name, open_mtx, high_mtx, low_mtx, close_mtx, belong, shenwan1, shenwan2, shenwan3, vol_mtx, amount_mtx = rschLib.loadDailyBarMtx("d:\\pklWeeklyUpdate\\")


init success


In [2]:
def logTime(s):
    print(s, datetime.datetime.now())

def generateTagAnalysis(tagName):
    print(tagName)
    tname = rschLib.tagMapper(tagName)['file']
    strategy_name = tname
    db.strategyBackTest.remove({'strategy_name':strategy_name})
    db.strategyBackTestTrades.remove({'strategy_name':strategy_name})
    db.strategyBackTest.update_one({'strategy_name': strategy_name}, {'$set':{
        'strategy_name':strategy_name
        }},upsert=True)
    startDate = 20180101
    if os.path.exists("d:\\pklWeeklyUpdate\\" + tname + ".pkl")==False:
        return -1
    with open("d:\\pklWeeklyUpdate\\" + tname + ".pkl", 'rb+') as f:
        tagInfo = pickle.load(f)
    tag_mtx = tagInfo['tag_mtx']
    off_start = rschLib.tagMapper(tagName)['off_start']
    rschLib.saveOffStart(strategy_name, off_start)
    tagNotNew = np.zeros(close_mtx.shape)
    for i in range(close_mtx.shape[0]):
        j = np.nonzero(close_mtx[i,:]>0)[0][0]
        v = np.min((close_mtx.shape[1], j+60))
        tagNotNew[i, v:]=1
    tag_mtx = (tag_mtx==1) & (tagNotNew==1) # 去掉新股票
    
    dtesUsed = dtes[dtes>=startDate]
    tag_mtxUsed = tag_mtx[:, dtes>=startDate]
    for (i, x) in enumerate(dtesUsed):
        q = name[tag_mtxUsed[:,i]==1]
        tk = tkrs[tag_mtxUsed[:,i]==1]
        query = []
        for i, y in enumerate(q):
            query.append({
                'strategy_name':strategy_name,
                'name':y,
                'ticker':tk[i],
                'dateIn':int(x)
            })
        if len(query)>0:
            db.strategyBackTestTrades.insert_many(query)
    rschLib.updateStrategyGeneratingStatus(strategy_name, '生成进度:10%。初始化标签。'+str(datetime.datetime.now()),10)
    rschLib.updateStrategyPrivacy(strategy_name, 'public')
    return 1

In [3]:
def analyzeStrategy(strategy_name, offStart, dtes, name, tkrs,maxD, inTime, otTime):
    timeAsFloat, timeLabels, maxM, dayOff, dayTimeAsFloat = rschLib.getTimeLabels(maxD)
    # get trades
    R = open_mtx[:, 1:]/close_mtx[:,:-1]-1  #使用收盘到开盘的回报率来修正分红和拆股
    R = np.hstack((np.zeros((R.shape[0],1)), R))
    logTime('getTradesFast')
    tradesUsed, r_withnan = rschLib.getTradesFast(strategy_name, name, tkrs, dtes, maxD, dayTimeAsFloat, R)
    if tradesUsed == -1:
        return
    logTime('getTradesFast done')
    # get trade samples by good/bad trades
    tradeArea=[inTime,otTime]
    r = r_withnan.copy()
    r[np.isfinite(r)==False]=0
    # draw price change
    idxTradable = np.isfinite(r_withnan[:,tradeArea[0]])
    logTime('getTradesAnalysisSampleGroups')
    result = rschLib.getTradeAnalysisSampleGroups(r, idxTradable, tradeArea)
    logTime('getTradesAnalysisSampleGroups done')
    logTime('drawPriceChange')
    rschLib.drawPriceChange(r[idxTradable,:], strategy_name, timeLabels=timeLabels, tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood10'], strategy_name, timeLabels=timeLabels, title='盈利前10%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rGood30'], strategy_name, timeLabels=timeLabels, title='盈利前30%交易', tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad10'], strategy_name, timeLabels=timeLabels, title='亏损前10%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rBad30'], strategy_name, timeLabels=timeLabels, title='亏损前30%交易',  tp=tradeArea, showGraph='no')
    rschLib.drawPriceChange(result['rDieting'], strategy_name, timeLabels=timeLabels, title='第一天跌停', tp=tradeArea, showGraph='no')
    logTime('drawPriceChange done')
    # analyze tags
    #rschLib.analyzeTradeTags(tradesUsed, result['rGood10'], result['idxGood10'], '盈利前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rGood30'], result['idxGood30'], '盈利前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rBad10'], result['idxBad10'], '亏损前10%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rBad30'], result['idxBad30'], '亏损前30%交易',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #rschLib.analyzeTradeTags(tradesUsed, result['rDieting'], result['idxDieting'], '第一天跌停',strategy_name, dtes, tkrs, offStart,"d:\\pklWeeklyUpdate\\")
    #get tag names
    tnames, tagNamesEn,t2 = rschLib.getTagNames()
    logTime('analyzeTradeTags')
    idxOverLapTagList=rschLib.analyzeTradeTags(tradesUsed, r, list(range(len(tradesUsed))), '所有交易',strategy_name, dtes, tkrs, offStart, "d:\\pklWeeklyUpdate\\")
    logTime('analyzeTradeTags done')
    #draw pnl and tag pnl
    importlib.reload(rschLib)
    logTime('getPnlFast')
    [dtesByTrade, pnlByTrade] = rschLib.getPnlFast(r, idxTradable, dtes, tkrs, name, tradesUsed, inTime, otTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    logTime('getPnlFast done')
    logTime('aggregatePnlAndDtes')
    [dtesPnlAggr,pnlAggr, numTrades] = rschLib.aggregatePnlAndDtes(dtesByTrade, pnlByTrade)
    logTime('aggregatePnlAndDtes done')
    rschLib.drawPNL(dtesPnlAggr, pnlAggr, dtes, strategy_name,showFigure='no', toDatabase='yes')
    for i in range(len(tnames)):
        tagName = tnames[i]
        [dtesWithTag, pnlWithTag,n] = rschLib.aggregatePnlAndDtes(dtesByTrade[idxOverLapTagList[i]],pnlByTrade[idxOverLapTagList[i]])
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name, showFigure='no', toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType=tagName)
        rschLib.drawPNL(dtesWithTag, pnlWithTag, dtes, strategy_name+'+'+tagNamesEn[i], showFigure='no',  toDatabase='yes', dateStart=dtesPnlAggr[0], pnlType='pnl')
        # 求非涨停，以及伴随有tagName标签的交易，绘制价格变化曲线
        idxTradableAndHasTag = list(np.intersect1d(np.nonzero(idxTradable)[0], np.array(idxOverLapTagList[i])))
        rschLib.drawPriceChange(r[idxTradableAndHasTag,:], strategy_name, timeLabels=timeLabels, title=tagName,  tp=tradeArea, showGraph='no')
    # control group and optimal group
    [rawInTime, rawOtTime] = rschLib.getDefaultTradeTime(offStart)
    idxTradableRaw = np.isfinite(r_withnan[:, rawInTime])
    [dtesByTradeRaw, pnlByTradeRaw] = rschLib.getPnlFast(r, idxTradableRaw, dtes, tkrs, name, tradesUsed, rawInTime, rawOtTime, dayOff, timeAsFloat, toDatabase='yes', strategy_name=strategy_name)
    [dtesPnlAggrRaw, pnlAggrRaw, numTradesRaw] = rschLib.aggregatePnlAndDtes(dtesByTradeRaw, pnlByTradeRaw)
    rschLib.drawPNL(dtesPnlAggrRaw, pnlAggrRaw, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='rawPnl')
    [bestInTime, bestOtTime] = rschLib.getOptimalTradeTime(r[idxTradable, :], rawInTime)
    idxTradableBestInTime = np.isfinite(r_withnan[:, bestInTime])
    [dtesByTradeBest, pnlByTradeBest] = rschLib.getPnlFast(r, idxTradableBestInTime, dtes, tkrs, name, tradesUsed, bestInTime, bestOtTime, dayOff, timeAsFloat, toDatabase='no', strategy_name=strategy_name)
    [dtesPnlAggrBest, pnlAggrBest, k] = rschLib.aggregatePnlAndDtes(dtesByTradeBest, pnlByTradeBest)
    rschLib.drawPNL(dtesPnlAggrBest, pnlAggrBest, dtes, strategy_name, showFigure='yes', toDatabase='yes', pnlType='optimalTradeTimePnl')
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimePoint', int(bestInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimePoint', int(bestOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'bestInTimeLabel', timeLabels[bestInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'bestOtTimeLabel', timeLabels[bestOtTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimePoint', int(rawInTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimePoint', int(rawOtTime))
    rschLib.updateStrategyBackTest(strategy_name, 'rawInTimeLabel', timeLabels[rawInTime])
    rschLib.updateStrategyBackTest(strategy_name, 'rawOtTimeLabel', timeLabels[rawOtTime])
    #analysis of number of trades vs performance
    importlib.reload(rschLib)
    rschLib.pnlVsNumtrades(pnlAggr, numTrades, strategy_name, toDatabase='yes')
    rschLib.saveOffStart(strategy_name, offStart)
    

In [4]:
importlib.reload(rschLib)
import os
strategy_names = [x['file'] for x in rschLib.tagDict().values()]
offStarts = [x['off_start'] for x in rschLib.tagDict().values()]
tagNames = list(rschLib.tagDict().keys())
maxD = 5


In [ ]:
%load_ext line_profiler
importlib.reload(rschLib)
for (i,strategy_name) in enumerate(strategy_names):
    m = list(db.strategyBackTest.find({'strategy_name':strategy_name}))
    if len(m)>0:
        q = m[0]
        flag = 1
        if ('statusCode' in q):
            if (q['statusCode']<100):
                flag = 0
        else:
            flag = 0
    else:
        flag = 0
    if flag==1:
        print(strategy_name, 'finished generating, pass')
        continue
    importlib.reload(rschLib)
    offStart = offStarts[i]
    [inTime, otTime] = rschLib.getDefaultTradeTime(offStart)
    flag = generateTagAnalysis(tagNames[i])
    if flag==1:
        analyzeStrategy(strategy_name, offStart, dtes, name, tkrs, maxD, inTime, otTime)
 

净利润增速连续三年大于20%


C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
  if __name__ == '__main__':
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in true_divide
  after removing the cwd from sys.path.
C:\ProgramData\Anaconda3\envs\research\lib\site-packages\ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in true_divide
  after removing the cwd from sys.path.


getTradesFast 2020-12-23 16:59:30.192951
从tradesUsed cache中踢出 20201214
从tradesUsed cache中踢出 20201211
从tradesUsed cache中踢出 20201210
从tradesUsed cache中踢出 20201209
204
tagNetProfitGrowth3YBiggerThan20P 最后完整日期 20201208 新增记录 204 总记录 71212
0 / 204 20201209


C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:448: RuntimeWarning: invalid value encountered in true_divide
  ro2c = closeList/openList - 1
C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:451: RuntimeWarning: invalid value encountered in true_divide
  rc2o[1:] = openList[1:]/closeList[:-1] - 1


getTradesFast done 2020-12-23 16:59:51.797085
getTradesAnalysisSampleGroups 2020-12-23 16:59:52.688527
getTradesAnalysisSampleGroups done 2020-12-23 16:59:53.454205
drawPriceChange 2020-12-23 16:59:53.455634
drawPriceChange done 2020-12-23 16:59:57.117745
analyzeTradeTags 2020-12-23 16:59:57.213466
['净利润增速连续三年大于20%', '季度净利润连续三年大于1亿', '跳空高开', '突破1年新高(开盘)', '突破3个月新高(开盘)', '均线多头排列', 'MACD的DIFF高于DEA', 'MACD的DIFF大于0', '交易量放大2倍', '流通市值大于100亿', '流通市值大于250亿', '流通市值大于500亿', '流通市值小于100亿', '流通市值小于50亿', '流通市值小于25亿', 'KDJ金叉', '十大股东持股比例小于20%', '十大股东持股比例20%~50%', '十大股东持股比例50%~80%', '十大股东持股比例大于80%', '交通运输', '休闲服务', '传媒', '公用事业', '农林牧渔', '化工', '医药生物', '商业贸易', '国防军工', '家用电器', '建筑材料', '建筑装饰', '房地产', '有色金属', '机械设备', '汽车', '电子', '电气设备', '纺织服装', '综合', '计算机', '轻工制造', '通信', '采掘', '钢铁', '银行', '非银金融', '食品饮料', '5日乖离率超买Ⅰ', '5日乖离率超买Ⅱ', '5日乖离率超卖Ⅰ', '3个月内涨幅>50%', '6个月内涨幅>50%', '1年内涨幅>50%', '5日乖离率超卖Ⅱ', '价格突破40周周线', '价格位于40周周线的0%-1%区间', '40周周线趋势向上', '不要SP500涨幅>2%', '不要SP500跌幅>2%']
所有交易 净利润增速连续三年大于20% 100.0
所有交易 季度净利润连

...

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:451: RuntimeWarning: divide by zero encountered in true_divide
  rc2o[1:] = openList[1:]/closeList[:-1] - 1


tagCirculateMarketValueBiggerThan100Y bulk write SH600675 20190731 20190806
tagCirculateMarketValueBiggerThan100Y bulk write SZ002304 20190802 20190808
260000 / 535177 20190805
tagCirculateMarketValueBiggerThan100Y bulk write SH600926 20190805 20190809
tagCirculateMarketValueBiggerThan100Y bulk write SZ002739 20190807 20190813
tagCirculateMarketValueBiggerThan100Y bulk write SH601668 20190808 20190814
tagCirculateMarketValueBiggerThan100Y bulk write SZ300676 20190812 20190816
tagCirculateMarketValueBiggerThan100Y bulk write SH603568 20190813 20190819
tagCirculateMarketValueBiggerThan100Y bulk write SH600021 20190814 20190820
tagCirculateMarketValueBiggerThan100Y bulk write SH603868 20190816 20190822
tagCirculateMarketValueBiggerThan100Y bulk write SH600029 20190819 20190823
tagCirculateMarketValueBiggerThan100Y bulk write SH603678 20190821 20190827
tagCirculateMarketValueBiggerThan100Y bulk write SH600161 20190822 20190828
270000 / 535177 20190823
tagCirculateMarketValueBiggerThan100Y 

C:\Users\xudong\Documents\github\coresearch\funcs\rschLib.py:422: RuntimeWarning: All-NaN axis encountered
  if np.nanmin(findCache['rList'])<-0.4:  # 不可能一分钟跌超过40%


tagCirculateMarketValueBiggerThan100Y bulk write SH600094 20191023 20191029
tagCirculateMarketValueBiggerThan100Y bulk write SZ000723 20191025 20191031
tagCirculateMarketValueBiggerThan100Y bulk write SH600305 20191028 20191101
tagCirculateMarketValueBiggerThan100Y bulk write SZ000878 20191030 20191105
300000 / 535177 20191031
tagCirculateMarketValueBiggerThan100Y bulk write SH600637 20191031 20191106
tagCirculateMarketValueBiggerThan100Y bulk write SZ002461 20191104 20191108
tagCirculateMarketValueBiggerThan100Y bulk write SH601800 20191105 20191111
tagCirculateMarketValueBiggerThan100Y bulk write SH600111 20191106 20191112
tagCirculateMarketValueBiggerThan100Y bulk write SZ000860 20191108 20191114
tagCirculateMarketValueBiggerThan100Y bulk write SH600755 20191111 20191115
tagCirculateMarketValueBiggerThan100Y bulk write SZ002714 20191113 20191119
tagCirculateMarketValueBiggerThan100Y bulk write SH603444 20191114 20191120
tagCirculateMarketValueBiggerThan100Y bulk write SH600332 20191

tagCirculateMarketValueBiggerThan100Y bulk write SH600885 20200525 20200529
tagCirculateMarketValueBiggerThan100Y bulk write SH600062 20200526 20200601
tagCirculateMarketValueBiggerThan100Y bulk write SZ002281 20200528 20200603
tagCirculateMarketValueBiggerThan100Y bulk write SH603858 20200529 20200604
tagCirculateMarketValueBiggerThan100Y bulk write SH600682 20200601 20200605
410000 / 535177 20200603
tagCirculateMarketValueBiggerThan100Y bulk write SZ300373 20200603 20200609
tagCirculateMarketValueBiggerThan100Y bulk write SZ002138 20200604 20200610
tagCirculateMarketValueBiggerThan100Y bulk write SH603658 20200605 20200611
tagCirculateMarketValueBiggerThan100Y bulk write SH600729 20200608 20200612
tagCirculateMarketValueBiggerThan100Y bulk write SH600037 20200609 20200615
tagCirculateMarketValueBiggerThan100Y bulk write SZ002411 20200611 20200617
tagCirculateMarketValueBiggerThan100Y bulk write SZ000598 20200612 20200618
tagCirculateMarketValueBiggerThan100Y bulk write SH601162 20200

In [ ]:
# importlib.reload(rschLib)
# %load_ext line_profiler
# %lprun -f generateTagAnalysis generateTagAnalysis(tagName)